In [1]:
import glob
import pandas as pd
import geopy.distance
import geopandas as gpd

# Initalize ddarung_master.csv

In [113]:
ddarung = pd.read_csv('../ddarung_master.csv', encoding = 'cp949')

ddarung['borrow_count_weekday'] = 0
ddarung['borrow_count_weekend'] = 0

ddarung['return_count_weekday'] = 0
ddarung['return_count_weekend'] = 0

print(ddarung.shape)
ddarung.head(1)

(2719, 29)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,No,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,...,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,borrow_count,return_count,balance_rate,borrow_count_weekday,borrow_count_weekend,return_count_weekday,return_count_weekend
0,241,0,0,0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,...,0,0,ST-117,27924,20197,-0.160574,0,0,0,0


In [108]:
bcycl_test = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_20221.csv', encoding = 'cp949')
#bcycl_test = bcycl_test.drop(columns = ['Unnamed: 0'])
#bcycl_test.columns = ['기준일자', '기준시간대', '시작_대여소', '종료_대여소', '전체건수', '전체_이용_분', '전체_이용_거리']
#bcycl_test = bcycl_test.rename(columns={'기준일자':'기준_날짜'})

bcycl_test.to_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_20221.csv', encoding = 'cp949')
print(bcycl_test.shape)
#bcycl_test = bcycl_test[bcycl_test['dayofweek'] == "weekend"]
#bcycl_test
bcycl_test['dayofweek'].unique()

(135698, 16)


array(['weekday'], dtype=object)

In [74]:
bcycl_test = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_20228.csv', encoding = 'utf8')

#bcycl_test = bcycl_test.drop(columns = ['Unnamed: 0'])
#bcycl_test.columns = ['기준_날짜', '기준_시간', ' ', '시작_대여소', '시작_대여소_주소', '종료_대여소', '종료_대여소_주소', '전체건수', '전체_이용_분', '전체_이용_거리']
bcycl_test.shape
#bcycl_test.to_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_20229.csv', encoding = 'utf8')
#len(tpss_bcycl.loc[tpss_bcycl['시작_대여소'] == row['대여소_ID']])

(6903282, 16)

In [94]:
len(bcycl_test.loc[(bcycl_test['시작_대여소'] == row['대여소_ID']) & (bcycl_test['dayofweek'] == "weekend")])
#len(bcycl_test.loc[bcycl_test['dayofweek'] == "weekday"])
#pd.unique(bcycl_test['dayofweek'])

653

In [63]:
ddarung.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,No,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,...,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,borrow_count,return_count,balance_rate,borrow_count_weekday,borrow_count_weekend,return_count_weekday,return_count_weekend
0,241,0,0,0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,...,0,0,ST-117,27924,20197,-0.160574,0,0,0,0
1,947,1,1,1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,...,0,0,ST-118,49420,47929,-0.015316,0,0,0,0
2,772,2,2,2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,...,0,0,ST-119,54335,51478,-0.027000,0,0,0,0
3,1065,3,3,3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,...,0,0,ST-121,33421,32736,-0.010354,0,0,0,0
4,1147,4,4,4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,...,0,3,ST-123,23216,22869,-0.007530,0,0,0,0


# count borrow, return count by month

In [111]:
from datetime import datetime
import numpy as np

dayofweek = [5,6]

for i in range(1, 2):
        
    if(i == 8 or i == 9):
        encoding = "utf8"
    else:
        encoding = "cp949"
        
    tpss_bcycl = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_2022%d.csv'%i, encoding = encoding)
    print(i , "," , tpss_bcycl.shape)
    tpss_bcycl['date'] = ""
    tpss_bcycl['date'] = pd.to_datetime(tpss_bcycl['기준_날짜'], format="%Y%m%d")
    
    tpss_bcycl['dayofweek'] = "" 
    tpss_bcycl['dayofweek'] = tpss_bcycl['date'].dt.weekday
    
    day_of_week_korean = np.array(['weekday', 'weekday', 'weekday', 'weekday', 'weekday', 'weekend', 'weekend'])
    tpss_bcycl['dayofweek'] = day_of_week_korean[tpss_bcycl['dayofweek']]
    
    tpss_bcycl.to_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_2022%d.csv'%i, encoding = encoding)

1 , (9290665, 13)


In [121]:
for i in range(1, 13):
        
    if(i == 8 or i == 9):
        encoding = "utf8"
    else:
        encoding = "cp949"
        
    tpss_bcycl = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_2022%d.csv'%i, encoding = encoding)
    print(i , "," , tpss_bcycl.shape)

    for index, row in ddarung.iterrows():
        ddarung['borrow_count_weekday'][index] = ddarung['borrow_count_weekday'][index] + 
        len(tpss_bcycl.loc[(tpss_bcycl['시작_대여소'] == row['대여소_ID']) & (tpss_bcycl['dayofweek'] == "weekday")])
        ddarung['borrow_count_weekend'][index] = ddarung['borrow_count_weekend'][index] + 
        len(tpss_bcycl.loc[(tpss_bcycl['시작_대여소'] == row['대여소_ID']) & (tpss_bcycl['dayofweek'] == "weekend")])

        ddarung['return_count_weekday'][index] = ddarung['return_count_weekday'][index] + 
        len(tpss_bcycl.loc[(tpss_bcycl['종료_대여소'] == row['대여소_ID']) & (tpss_bcycl['dayofweek'] == "weekday")])
        ddarung['return_count_weekend'][index] = ddarung['return_count_weekend'][index] + 
        len(tpss_bcycl.loc[(tpss_bcycl['종료_대여소'] == row['대여소_ID']) & (tpss_bcycl['dayofweek'] == "weekend")])
        
        if(index % 100 == 0):
            print(i, "/ 13  ", index, "/" , ddarung.shape[0])

10 , (8526697, 17)
10 / 13   0 / 2719
10 / 13   100 / 2719
10 / 13   200 / 2719
10 / 13   300 / 2719
10 / 13   400 / 2719
10 / 13   500 / 2719
10 / 13   600 / 2719
10 / 13   700 / 2719
10 / 13   800 / 2719
10 / 13   900 / 2719
10 / 13   1000 / 2719
10 / 13   1100 / 2719
10 / 13   1200 / 2719
10 / 13   1300 / 2719
10 / 13   1400 / 2719
10 / 13   1500 / 2719
10 / 13   1600 / 2719
10 / 13   1700 / 2719
10 / 13   1800 / 2719
10 / 13   1900 / 2719
10 / 13   2000 / 2719
10 / 13   2100 / 2719
10 / 13   2200 / 2719
10 / 13   2300 / 2719
10 / 13   2400 / 2719
10 / 13   2500 / 2719
10 / 13   2600 / 2719
10 / 13   2700 / 2719
11 , (6838458, 15)
11 / 13   0 / 2719
11 / 13   100 / 2719
11 / 13   200 / 2719
11 / 13   300 / 2719
11 / 13   400 / 2719
11 / 13   500 / 2719
11 / 13   600 / 2719
11 / 13   700 / 2719
11 / 13   800 / 2719
11 / 13   900 / 2719
11 / 13   1000 / 2719
11 / 13   1100 / 2719
11 / 13   1200 / 2719
11 / 13   1300 / 2719
11 / 13   1400 / 2719
11 / 13   1500 / 2719
11 / 13   1600 / 2

In [129]:
#SAVE
ddarung.to_csv('../ddarung_master.csv', encoding = 'cp949')

In [128]:
ddarung.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,No,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,...,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,borrow_count,return_count,balance_rate,borrow_count_weekday,borrow_count_weekend,return_count_weekday,return_count_weekend
0,241,0,0,0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,...,0,0,ST-117,25407,18175,0.165940,19585,5822,13810,4365
1,947,1,1,1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,...,0,0,ST-118,45075,43549,0.017219,32954,12121,31643,11906
2,772,2,2,2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,...,0,0,ST-119,49593,46961,0.027259,42275,7318,39937,7024
3,1065,3,3,3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,...,0,0,ST-121,29763,29008,0.012846,25322,4441,24600,4408
4,1147,4,4,4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,...,0,3,ST-123,21094,20778,0.007547,17143,3951,17035,3743


In [127]:
ddarung['balance_rate'] = (ddarung['borrow_count'] - ddarung['return_count']) / (ddarung['return_count'] + ddarung['borrow_count'])

In [125]:
ddarung['return_count'] = ddarung['return_count_weekday'] + ddarung['return_count_weekend']

In [4]:
import warnings
warnings.filterwarnings(action='ignore')


for i in range(1, 13):
    
    if(i == 8 or i == 9):
        encoding = "utf8"
    else:
        encoding = "cp949"
        
    tpss_bcycl = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_2022%d.csv'%i, encoding = encoding)
    print(i , "," , tpss_bcycl.shape)

    for index, row in tpss_bcycl.iterrows():

        borrow_station = str(row['시작_대여소'])[0:]
        return_station = str(row['종료_대여소'])[0:]

        borrow_index = ddarung[ddarung['대여소_ID'] == borrow_station]['No']
        return_index = ddarung[ddarung['대여소_ID'] == return_station]['No']

        ddarung['borrow_count'][borrow_index] = ddarung['borrow_count'][borrow_index] + 1
        ddarung['return_count'][return_index] = ddarung['return_count'][return_index] + 1

        if(index % 10000 == 0):
            print(i, "/ 12", round((i-1)/12*100,3), "%")
            print(index, "/", tpss_bcycl.shape[0], round(index/tpss_bcycl.shape[0],3) * 100, "%")

ddarung.head()

1 , (2815570, 8)
1 /12 0.0 %
0 / 2815570 0.0 %


KeyboardInterrupt: 